<a href="https://colab.research.google.com/github/LEEGWONSEOK/CSAI_assignment/blob/main/%5B%EA%B7%B9%EB%9D%BD%5D_n222a_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
!pip install --upgrade category_encoders

#### **01. Domain & Data Preprocessing**

#### 데이터 구조
- 데이터 : 
- 관측치 : 
- 변수 : 

#### 설명변수(features) : 예측값을 설명할 수 있는 변수
- ff


#### 반응변수(targets) : 예측하고자 하는 값
- **vacc_h1n1_f** : 백신 맞음(1) / 안 맞음(0)

In [ ]:
## 모듈 불러오기

# 데이터 전처리
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 데이터 시각화
import matplotlib.pyplot as plt

# 기계학습 모델 및 평가
#from sklearn.neighb
from sklearn.metrics import accuracy_score, recall_score, precision_score, mean_squared_error, mean_absolute_error, r2_score

# 경고문 제거
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
## 데이터 불러오기

train_labels_data = 'https://raw.githubusercontent.com/LEEGWONSEOK/Kaggle01_Prediction-of-H1N1-vaccination/main/_dataset/train_labels.csv'
train_data = 'https://raw.githubusercontent.com/LEEGWONSEOK/Kaggle01_Prediction-of-H1N1-vaccination/main/_dataset/train.csv'
test_data = 'https://raw.githubusercontent.com/LEEGWONSEOK/Kaggle01_Prediction-of-H1N1-vaccination/main/_dataset/test.csv'
sample_sub_data = 'https://raw.githubusercontent.com/LEEGWONSEOK/Kaggle01_Prediction-of-H1N1-vaccination/main/_dataset/submission.csv'

target = 'vacc_h1n1_f'
#target = 'vacc_seas_f'

train = pd.merge(pd.read_csv(train_data), pd.read_csv(train_labels_data)[target], left_index=True, right_index=True)
test = pd.read_csv(test_data)
sample_submission = pd.read_csv(sample_sub_data)

####**02. Missing value 확인**

In [ ]:
## 결측치 확인

train.T.duplicated()
# 결측치가 없는 것을 확인할 수 있다.

h1n1_concern                   False
h1n1_knowledge                 False
behavioral_antiviral_meds      False
behavioral_avoidance           False
behavioral_face_mask           False
behavioral_wash_hands          False
behavioral_large_gatherings    False
behavioral_outside_home        False
behavioral_touch_face          False
doctor_recc_h1n1               False
doctor_recc_seasonal           False
chronic_med_condition          False
child_under_6_months           False
health_insurance               False
health_worker                  False
opinion_h1n1_vacc_effective    False
opinion_h1n1_risk              False
opinion_h1n1_sick_from_vacc    False
opinion_seas_vacc_effective    False
opinion_seas_risk              False
opinion_seas_sick_from_vacc    False
agegrp                         False
education_comp                 False
raceeth4_i                     False
sex_i                          False
inc_pov                        False
marital                        False
r

In [ ]:
train.describe(exclude='number')

,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,agegrp,employment_status,census_msa,employment_industry,employment_occupation,state
count,33375,33329,33285,33220,33153,33089,42154,31876,42154,17165,17165,42154
unique,6,6,6,6,6,6,7,3,3,23,25,51
top,Somewhat Effective,Somewhat Low,Not Very Worried,Somewhat Effective,Somewhat Low,Not At All Worried,65+ Years,Employed,"MSA, Not Principle City",Health Care and Social Assistance Industries,Management Occupations,NEW MEXICO
freq,14729,12520,11582,14543,11397,14749,8517,17173,18957,3086,2188,1017


In [ ]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
employment_status,31876,3,Employed,17173
census_msa,42154,3,"MSA, Not Principle City",18957
opinion_h1n1_vacc_effective,33375,6,Somewhat Effective,14729
opinion_h1n1_risk,33329,6,Somewhat Low,12520
opinion_h1n1_sick_from_vacc,33285,6,Not Very Worried,11582
opinion_seas_vacc_effective,33220,6,Somewhat Effective,14543
opinion_seas_risk,33153,6,Somewhat Low,11397
opinion_seas_sick_from_vacc,33089,6,Not At All Worried,14749
agegrp,42154,7,65+ Years,8517
employment_industry,17165,23,Health Care and Social Assistance Industries,3086


#### **03. 데이터 세트 분리(train / valid / test)**

In [ ]:
train, val = train_test_split(train, train_size=0.8, test_size=0.2,
                              stratify=train[target], random_state=2)
train.shape, val.shape, test.shape  # : ((33723, 39), (8431, 39), (28104, 38))

((33723, 39), (8431, 39), (28104, 38))

#### **04. train[target] 비율 확인(기준모델)**

In [ ]:
train[target].value_counts(normalize=True)

# 백신 맞음 : 24% / 안 맞음 : 76% → 클래스(0,1)이 불균형하다.

0    0.760935
1    0.239065
Name: vacc_h1n1_f, dtype: float64

#### **05. Features Engineering**

In [ ]:
import numpy as np

def engineer(df):
    # 높은 카디널리티를 가지는 특성 제거.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique()
    selected_features = labels[labels <= 30].index.tolist() # 30보다 작은 특성 선택
    df = df[selected_features]

    # 새로운 특성 생성
    behaviorals = [col for col in df.columns if 'behavioral' in col]
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)

    return df

train = engineer(train)
val = engineer(val)
test = engineer(test)

features = train.drop(columns=[target]).columns

#### **06. 세트별 target/features 분리**

In [ ]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

#### **07-1. 랜덤포레스트 모델링(One-hot Encoder)**

In [ ]:
## 걸리는 시간

%%time

## 모듈
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('훈련 정확도(One-hot Encoder) : ', pipe.score(X_train, y_train))
print('검증 정확도(One-hot Encoder) : ', pipe.score(X_val, y_val))

훈련 정확도(One-hot Encoder) :  0.990837114135753
검증 정확도(One-hot Encoder) :  0.8270667773692326
CPU times: user 11 s, sys: 198 ms, total: 11.2 s
Wall time: 6.74 s


#### **07-2. 랜덤포레스트 모델링(Ordinal Encoder)**

In [ ]:
%%time
from category_encoders import OrdinalEncoder

pipe_ord = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe_ord.fit(X_train, y_train)
print('훈련 정확도(Ordinal Encoder) : ', pipe_ord.score(X_train, y_train))
print('검증 정확도(Ordinal Encoder) : ', pipe_ord.score(X_val, y_val))

훈련 정확도(Ordinal Encoder) :  0.9908667674880646
검증 정확도(Ordinal Encoder) :  0.8239829201755426
CPU times: user 9.12 s, sys: 146 ms, total: 9.27 s
Wall time: 5.59 s


#### **07-3. Ordinal - GridsearchCV 적용**

In [ ]:
## gridsearchCV

## 모듈
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline   # 'key' 값이 필요해서 make-pipeline말고 그냥 요놈 써봤습니다.

In [ ]:
pipe_ord = Pipeline([
    ('ordinal',OrdinalEncoder()), 
    ('imputer', SimpleImputer()), 
    ('rf', RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True, class_weight='balanced'))
])

param_grid = {
    #'rf__n_estimators': [10, 20, 30],
    'rf__min_samples_leaf': [5, 10, 15],
    'rf__max_depth': [7, 8, 9],
    'rf__criterion': ['gini', 'entropy'],
    'rf__max_features': ['auto', 'sqrt', 'log2']
}

CV_pipe_ord = GridSearchCV(pipe_ord, param_grid=param_grid, cv=10, verbose=1, n_jobs=-1)
CV_pipe_ord.fit(X_train, y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('ordinal', OrdinalEncoder()),
                                       ('imputer', SimpleImputer()),
                                       ('rf',
                                        RandomForestClassifier(n_jobs=-1,
                                                               oob_score=True,
                                                               random_state=10))]),
             n_jobs=-1,
             param_grid={'rf__criterion': ['gini', 'entropy'],
                         'rf__max_depth': [7, 8, 9],
                         'rf__max_features': ['auto', 'sqrt', 'log2'],
                         'rf__min_samples_leaf': [5, 10, 15]},
             verbose=1)

In [ ]:
CV_pipe_ord.best_params_

{'rf__criterion': 'gini',
 'rf__max_depth': 9,
 'rf__max_features': 'auto',
 'rf__min_samples_leaf': 5}

하이퍼 파라미터

- criterion : 'gini'
- max_depth : 9
- max_features': 'auto'
- min_samples_leaf : 5

값들이 가장 높게 나온다는 것을 알 수 있다.

In [ ]:
result_table = pd.DataFrame(CV_pipe_ord.cv_results_)
result_table = result_table.sort_values(by='mean_test_score', ascending=False)
print(result_table[['params', 'mean_test_score']])
#print(result_table['mean_test_score'])

                                               params  mean_test_score
18  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.819826
21  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.819826
46  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.819293
49  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.819293
45  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.818966
48  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.818966
23  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.818877
20  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.818877
47  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.818818
50  {'rf__criterion': 'entropy', 'rf__max_depth': ...         0.818818
22  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.818788
19  {'rf__criterion': 'gini', 'rf__max_depth': 9, ...         0.818788
9   {'rf__criterion': 'gini', 'rf__max_depth': 8, ...         0.818225
12  {'

In [ ]:
pipe_ord2 = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(criterion= 'gini', 
                           max_depth= 9, 
                           max_features= 'auto', 
                           min_samples_leaf= 5,
                           random_state= 10,
                           n_jobs= -1,
                           oob_score=True)
)

pipe_ord2.fit(X_train, y_train)
print('훈련 정확도(Ordinal Encoder) : ', pipe_ord2.score(X_train, y_train))
print('검증 정확도(Ordinal Encoder) : ', pipe_ord2.score(X_val, y_val))

훈련 정확도(Ordinal Encoder) :  0.8315986122231118
검증 정확도(Ordinal Encoder) :  0.8286087059660776


In [ ]:
predict = pipe_ord2.predict(X_test)
predict

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
sample_submission['vacc_h1n1_f'] = predict

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

결과적으로 검증 정확도가 조금 증가했다는 것을 확인할 수 있었다.